In [2]:
import os
import glob
import shutil
import re
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
src_folder = r'/awlab/projects/2015_08_Senescence/plates/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67'

# img_des_folder = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/plate_2017017086_ki67/images'
# mask_des_folder = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/plate_2017017086_ki67/masks'

img_des_folder = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/p2017017086_ki67_merge/images'
mask_des_folder = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/p2017017086_ki67_merge/masks'

file_prefix = 'CH_171219_Vh31_Goldilocks_plate_2017017086_ki67'

# Collect all images

In [3]:
def get_file_list(root_path, pattern='*.png'):
    # Method 1: os.walk
#     img_list = []
#     for root, dirs, files in os.walk(plate_path_root):
#         for f in files:
#             if '.png' in f:
#                 img_list.append(os.path.join(root,f))

    # Method 2: glob
    img_list = [f for f in glob.glob(os.path.join(root_path,'**',pattern), recursive=True)]
    
    return sorted(img_list)

In [ ]:
img_list = get_file_list(src_folder, pattern='*-[23].png')

In [ ]:
img_list[:5]

## Incucyte


In [6]:
img_src = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/raw/2018028010_PC9_nuclear_dye_test/Phase'
mask_src = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/raw/2018028010_PC9_nuclear_dye_test/RFP_mask'

img_des = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2018028010_PC9_nuclear_dye_test/images'
mask_des = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2018028010_PC9_nuclear_dye_test/masks'

# img_src = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/raw/2019028023_PC9_A549_with_nuclear_marker/Phase'
# mask_src = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/raw/2019028023_PC9_A549_with_nuclear_marker/RFP_mask'

# img_des = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2019028023_PC9_A549_with_nuclear_marker/images'
# mask_des = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2019028023_PC9_A549_with_nuclear_marker/masks'

file_prefix = '2018028010_PC9_nuclear_dye_test'

img_list = get_file_list(img_src, pattern='*.tif')
mask_list = get_file_list(mask_src, pattern='*.tif')


def save_to_png(img_list, des_dir):
    if not os.path.isdir(des_dir):
        os.makedirs(des_dir)
        
    N = len(img_list)
    for i, f in enumerate(img_list):
        print('{}/{}'.format(i+1, N), end='\r')
        
        img = cv2.imread(f)[:,:,0] # gray scale        
        new_name = os.path.join(des_dir, os.path.basename(f).split('.')[0] + '.png')
        cv2.imwrite(new_name, img)

In [7]:
save_to_png(img_list, img_des)
save_to_png(mask_list, mask_des)

### Merge nucleus and cell images

In [ ]:
def get_merged_image(img_grp, nuc_tag='-2.png', cell_tag='-3.png', max_level=4095.):
    """green: nucleus, red: cell"""
    nuc_img = img_grp + nuc_tag
    cell_img = img_grp + cell_tag
    
    nuc = np.float32(cv2.imread(nuc_img, cv2.IMREAD_UNCHANGED)) / max_level
    cell = np.float32(cv2.imread(cell_img, cv2.IMREAD_UNCHANGED)) / max_level
    b = np.zeros_like(nuc)

    img = np.clip(cv2.merge([cell, nuc, b]), 0., 1.)
    img = np.uint8(img * 255.)
    
    return img    

### Copy to destination folder

#### For separate images

In [ ]:
if not os.path.isdir(img_des_folder):
    os.makedirs(img_des_folder)

N = len(img_group)
for i, f in enumerate(img_list):
    print('{}/{}'.format(i+1, N), end='\r')
    new_path = os.path.join(img_des_folder, os.path.basename(f))
    shutil.copyfile(f, new_path)

#### For merged images

In [ ]:
if not os.path.isdir(img_des_folder):
    os.makedirs(img_des_folder)

img_group = sorted(list(set([g.split('-')[0] for g in img_list])))
N = len(img_group)
for i, g in enumerate(img_group):
    print('{}/{}'.format(i+1, N), end='\r')
    
    img = get_merged_image(g)    
    new_path = os.path.join(img_des_folder, os.path.basename(g) + '.png')
    cv2.imwrite(new_path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Get mask from .mat files

In [ ]:
mat_files = get_file_list(src_folder, pattern='segmentation-*.mat')
print(len(mat_files))
mat_files[:3]

In [ ]:
def get_well_name(file_name, pattern='segmentation-(?P<row>\d+)-(?P<col>\d+)'):
    m = re.match(pattern, os.path.basename(file_name))
    r_idx = int(m.group('row'))
    well = chr(r_idx+64) + m.group('col')
#     well = chr(r_idx+64) + '{:02d}'.format(int(m.group('col')))

    return well

In [ ]:
mat = scipy.io.loadmat(mat_files[0], struct_as_record=False, squeeze_me=True)
seg = mat['segmentation']
s = seg[0]

nuc_mask = s.dnamask>0
nuc_mask = np.uint8(nuc_mask*255.)
cell_mask = np.uint8(s.cellmask*255.)

In [ ]:
np.unique(cell_mask)

### Save masks to the destination folder

In [ ]:
MODE = 'Merge' # Separate or Merge

if not os.path.isdir(mask_des_folder):
    os.makedirs(mask_des_folder)
    
for i, f in enumerate(mat_files):
    print('{}/{}'.format(i+1, len(mat_files)), end='\r')
    
    # 1. load the mat file
    mat = scipy.io.loadmat(f, struct_as_record=False, squeeze_me=True)  
    well = get_well_name(f) # get well name
    
    # 2. Get the segmentation struct
    seg = mat['segmentation']

    # 3. Loop through each frame
    for j, s in enumerate(seg):
        nuc_mask = s.dnamask>0
        nuc_mask = np.uint8(nuc_mask*255.)
        cell_mask = np.uint8(s.cellmask*255.)
        
        # 4. Form new file name
        frame_str = '{:04d}'.format(j)
        new_prefix = '_'.join([file_prefix, well, frame_str])
        
        if MODE == 'Separate':
            fname_nuc = os.path.join(mask_des_folder, new_prefix + '_nucleus.png')
            fname_cell = os.path.join(mask_des_folder, new_prefix + '_cell.png')

            # 5. Save to the mask folder
            cv2.imwrite(fname_nuc, nuc_mask)
            cv2.imwrite(fname_cell, cell_mask)
            
        elif MODE == 'Merge':
            cell_mask[nuc_mask>0] = 0.
            b = np.ones_like(nuc_mask) * 255.
            b[(nuc_mask>0) | (cell_mask>0)] = 0.
            b = np.uint8(b)
            mask = cv2.merge([b, nuc_mask, cell_mask])
            
            fname = os.path.join(mask_des_folder, new_prefix + '.png')            
            cv2.imwrite(fname, mask)

# Incucyte

In [ ]:
img_src = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/raw/2019028023_PC9_A549_with_nuclear_marker/Phase'
mask_src = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/raw/2019028023_PC9_A549_with_nuclear_marker/RFP_mask'

img_des = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2019028023_PC9_A549_with_nuclear_marker/images'
mask_des = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/2019028023_PC9_A549_with_nuclear_marker/masks'

file_prefix = '2019028023_PC9_A549_with_nuclear_marker'

img_list = get_file_list(img_src, pattern='*.tif')
mask_list = get_file_list(mask_src, pattern='*.tif')


def save_to_png(img_list, des_dir):
    if not os.path.isdir(des_dir):
        os.makedirs(des_dir)
        
    N = len(img_list)
    for i, f in enumerate(img_list):
        print('{}/{}'.format(i+1, N), end='\r')
        
        img = cv2.imread(f)[:,:,0] # gray scale        
        new_name = os.path.join(des_dir, os.path.basename(f).split('.')[0] + '.png')
        cv2.imwrite(new_name, img)

In [ ]:
save_to_png(img_list, img_des)
save_to_png(mask_list, mask_des)